# AI Dataset EDA Notebook (NumPy, pandas, Matplotlib, Seaborn)

এই নোটবুকটি স্বয়ংক্রিয়ভাবে তৈরি করা হয়েছে। এতে **NumPy, pandas, Matplotlib, Seaborn** ব্যবহার করে 
আপনার ডেটাসেটের উপর একটি বেসিক EDA টেমপ্লেট দেওয়া আছে।

## কীভাবে চালাবেন
1. নীচের সেলগুলো ক্রমানুসারে রান করুন।
2. `DATA_PATH` ভ্যারিয়েবলটি আপনার ডেটা ফাইল লোকেশনের সাথে মিলিয়ে নিন।

এই নোটবুক তৈরির সময় পাওয়া ফাইল তালিকা (zip থেকে extract করা):  
- `Cars Datasets 2025.csv`

> **Note:** এই নোটবুক ধরে নিচ্ছে আপনার ডেটা ফাইলটি লোকাল ডিরেক্টরিতে আছে।


In [ ]:

# Basic imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Render settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
# seaborn style (optional)
sns.set()


In [ ]:

# === CONFIG ===
# আপনার ডেটা ফাইলের লোকেশন দিন (নীচের লাইনটি প্রয়োজনে এডিট করুন)
DATA_PATH = r"""/mnt/data/dataset/Cars Datasets 2025.csv"""

# CSV/XLSX/XLS অনুযায়ী পড়ুন
file_ext = os.path.splitext(DATA_PATH)[1].lower()
if file_ext == ".csv":
    df = pd.read_csv(DATA_PATH)
elif file_ext in [".xlsx", ".xls"]:
    # যদি Multiple sheets থাকে, প্রথম শিট নেওয়া হলো; প্রয়োজনে sheet_name পরিবর্তন করুন
    df = pd.read_excel(DATA_PATH, sheet_name=0)
else:
    raise ValueError("Unsupported file extension: " + file_ext)

print("Shape:", df.shape)
df.head()


In [ ]:

# ডেটাসেট ইনফো
print("\n--- DataFrame Info ---")
print(df.info())

# বেসিক ডেসক্রিপটিভ স্ট্যাটস
display(df.describe(include='all').transpose())

# Missing values overview
missing = df.isna().sum().sort_values(ascending=False)
print("\n--- Missing Values (count) ---")
display(missing.to_frame("missing_count").head(20))


In [ ]:

# === NumPy Examples ===
# Numeric columns detect
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
print("Numeric columns:", numeric_cols)

if numeric_cols:
    # উদাহরণ: প্রথম numeric কলামের উপর NumPy দিয়ে কিছু হিসাব
    col = numeric_cols[0]
    arr = df[col].dropna().to_numpy()

    print(f"Column: {col}")
    print("Mean (np):", np.mean(arr))
    print("Std  (np):", np.std(arr))
    print("Min/Max (np):", np.min(arr), np.max(arr))

    # একটি সহজ transform উদাহরণ
    z = (arr - np.mean(arr)) / (np.std(arr) if np.std(arr) != 0 else 1)
    print("Z-score (first 10):", z[:10])
else:
    print("No numeric columns found for NumPy demo.")


In [ ]:

# === Matplotlib Example ===
import matplotlib.pyplot as plt

if numeric_cols:
    plt.figure(figsize=(6,4))
    col = numeric_cols[0]
    plt.hist(df[col].dropna(), bins=30)
    plt.title(f"Histogram of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()
else:
    print("No numeric column to plot with Matplotlib.")


In [ ]:

# === Seaborn Examples ===
import seaborn as sns

if len(numeric_cols) >= 2:
    # Pairplot on first few numeric columns
    subset = numeric_cols[:4]
    sns.pairplot(df[subset].dropna())
    plt.show()

if len(numeric_cols) >= 1:
    # Correlation heatmap
    corr = df[numeric_cols].corr(numeric_only=True)
    plt.figure(figsize=(6,5))
    sns.heatmap(corr, annot=False)
    plt.title("Correlation Heatmap (numeric columns)")
    plt.show()


In [ ]:

# === Categorical Overview ===
cat_cols = df.select_dtypes(include=["object","category","bool"]).columns.tolist()
print("Categorical columns:", cat_cols)

if cat_cols:
    col = cat_cols[0]
    print(f"Top categories in '{col}':")
    display(df[col].value_counts().head(20))
else:
    print("No categorical columns detected.")


In [ ]:

# === Simple Cleaning Examples ===
# Missing value imputation for numeric columns (median)
df_clean = df.copy()
for col in numeric_cols:
    if df_clean[col].isna().any():
        df_clean[col].fillna(df_clean[col].median(), inplace=True)

# For categoricals, fill with mode
for col in cat_cols:
    if df_clean[col].isna().any():
        df_clean[col].fillna(df_clean[col].mode(dropna=True).iloc[0] if not df_clean[col].mode(dropna=True).empty else "Unknown", inplace=True)

print("Original shape:", df.shape, " | Cleaned shape:", df_clean.shape)
df_clean.head()


In [ ]:

# Cleaned sample save (optional)
out_path = "cleaned_sample.csv"
df_clean.to_csv(out_path, index=False)
print("Saved cleaned sample to:", out_path)


## Next Steps
- প্রয়োজন অনুযায়ী ফিচার ইঞ্জিনিয়ারিং, আউটলায়ার হ্যান্ডলিং, স্কেলিং ইত্যাদি যোগ করুন।
- মডেলিং করতে চাইলে `scikit-learn` যোগ করে train/test split, baseline model ইত্যাদি করতে পারেন।
- ভিজুয়ালাইজেশনের জন্য আরও প্লট (boxplot, violinplot, barplot ইত্যাদি) যুক্ত করুন।
